In [1]:
import pandas as pd
import re

In [2]:
doc = "Web_access_log-akumenius.com.txt"

In [10]:
# Open log registry with a regex that separates all 
# data categories into different columns

ds = pd.read_csv(doc, 
                 sep = r'\s(?=(?:[^"]*"[^"]*")*[^"]*$)(?![^\[]*\])', 
                 engine='python',
                 na_values='-',
                 usecols=[0, 1, 4, 5, 6, 7, 8, 9],
                 header=None,
                 names=['server', 'ip', 'time', 'request', 'status', 'size', 'referer', 'user_agent'],
                )

In [11]:
ds.tail(10)

,server,ip,time,request,status,size,referer,user_agent
261863,akumenius.com,5.255.253.53,[02/Mar/2014:03:05:32 +0100],"""GET /robots.txt HTTP/1.1""",301,301.0,"""-""","""Mozilla/5.0 (compatible; YandexBot/3.0; +http..."
261864,akumenius.es,5.255.253.53,[02/Mar/2014:03:05:33 +0100],"""GET /robots.txt HTTP/1.1""",301,304.0,"""-""","""Mozilla/5.0 (compatible; YandexBot/3.0; +http..."
261865,akumenius.com,5.255.253.53,[02/Mar/2014:03:05:35 +0100],"""GET / HTTP/1.1""",301,301.0,"""-""","""Mozilla/5.0 (compatible; YandexBot/3.0; +http..."
261866,www.akumenius.com,5.255.253.53,[02/Mar/2014:03:05:34 +0100],"""GET / HTTP/1.1""",200,7528.0,"""-""","""Mozilla/5.0 (compatible; YandexBot/3.0; +http..."
261867,akumenius.es,5.255.253.53,[02/Mar/2014:03:05:35 +0100],"""GET / HTTP/1.1""",301,304.0,"""-""","""Mozilla/5.0 (compatible; YandexBot/3.0; +http..."
261868,www.akumenius.com,5.255.253.53,[02/Mar/2014:03:05:39 +0100],"""GET / HTTP/1.1""",200,7528.0,"""-""","""Mozilla/5.0 (compatible; YandexBot/3.0; +http..."
261869,www.akumenius.com,74.86.158.107,[02/Mar/2014:03:09:52 +0100],"""HEAD / HTTP/1.1""",200,NaN,"""-""","""Mozilla/5.0+(compatible; UptimeRobot/2.0; htt..."
261870,localhost,127.0.0.1,[02/Mar/2014:03:10:18 +0100],"""OPTIONS * HTTP/1.0""",200,NaN,"""-""","""Apache (internal dummy connection)"""
261871,localhost,127.0.0.1,[02/Mar/2014:03:10:18 +0100],"""OPTIONS * HTTP/1.0""",200,NaN,"""-""","""Apache (internal dummy connection)"""
261872,localhost,127.0.0.1,[02/Mar/2014:03:10:18 +0100],"""OPTIONS * HTTP/1.0""",200,NaN,"""-""","""Apache (internal dummy connection)"""


In [12]:
# Clean dataset by eliminating ""

ds["request"] = ds["request"].str.extract("([^\"]+)")

ds["referer"] = ds["referer"].str.extract("([^\"]+)")

ds["user_agent"] = ds["user_agent"].str.extract("([^\"]+)")

ds.tail(5)

,server,ip,time,request,status,size,referer,user_agent
261868,www.akumenius.com,5.255.253.53,[02/Mar/2014:03:05:39 +0100],GET / HTTP/1.1,200,7528.0,-,Mozilla/5.0 (compatible; YandexBot/3.0; +http:...
261869,www.akumenius.com,74.86.158.107,[02/Mar/2014:03:09:52 +0100],HEAD / HTTP/1.1,200,NaN,-,Mozilla/5.0+(compatible; UptimeRobot/2.0; http...
261870,localhost,127.0.0.1,[02/Mar/2014:03:10:18 +0100],OPTIONS * HTTP/1.0,200,NaN,-,Apache (internal dummy connection)
261871,localhost,127.0.0.1,[02/Mar/2014:03:10:18 +0100],OPTIONS * HTTP/1.0,200,NaN,-,Apache (internal dummy connection)
261872,localhost,127.0.0.1,[02/Mar/2014:03:10:18 +0100],OPTIONS * HTTP/1.0,200,NaN,-,Apache (internal dummy connection)


In [6]:
# To put IPs to use, import IPInfo Library; insert Token ID 

import ipinfo
handler = ipinfo.getHandler("6ec2f727d48c66")

In [7]:
# Convert IPs into a usable format

ipSize = ds.groupby("ip").size() # How many times an IP appears
series = ipSize.index # Different IPs, no duplicates

print(series)

Index(['101.199.108.59', '101.226.167.226', '107.178.37.48', '107.178.38.221',
       '107.21.14.116', '107.6.95.11', '107.6.95.122', '107.6.95.13',
       '107.6.95.16', '107.6.95.20',
       ...
       '95.62.55.142', '95.62.63.204', '95.63.1.210', '95.63.165.105',
       '95.63.2.45', '95.63.234.126', '95.63.60.249', '96.242.31.129',
       '98.137.206.41', '98.244.10.248'],
      dtype='object', name='ip', length=2921)


In [13]:
# .getBatchDetails() gets all data from an array of IPs

details = handler.getBatchDetails(series)

# Conversion of IP info dictionary into a DataFrame:

dataBase = pd.DataFrame.from_dict(details, orient = 'index')
dataBase.head(10)

,ip,city,region,country,loc,timezone,country_name,latitude,longitude,org,hostname,postal,bogon
101.199.108.59,101.199.108.59,Beijing,Beijing,CN,"39.9075,116.3972",Asia/Shanghai,China,39.9075,116.3972,NaN,NaN,NaN,NaN
101.226.167.226,101.226.167.226,Shanghai,Shanghai,CN,"31.2222,121.4581",Asia/Shanghai,China,31.2222,121.4581,AS4812 China Telecom (Group),NaN,NaN,NaN
107.178.37.48,107.178.37.48,San Jose,California,US,"37.3394,-121.8950",America/Los_Angeles,United States,37.3394,-121.8950,AS174 Cogent Communications,107-178-37-48.ip.cloudmosa.com,95103,NaN
107.178.38.221,107.178.38.221,San Jose,California,US,"37.3394,-121.8950",America/Los_Angeles,United States,37.3394,-121.8950,AS174 Cogent Communications,107-178-38-221.ip.cloudmosa.com,95103,NaN
107.21.14.116,107.21.14.116,Ashburn,Virginia,US,"39.0437,-77.4875",America/New_York,United States,39.0437,-77.4875,"AS14618 Amazon.com, Inc.",ec2-107-21-14-116.compute-1.amazonaws.com,20149,NaN
107.6.95.11,107.6.95.11,San Jose,California,US,"37.3394,-121.8950",America/Los_Angeles,United States,37.3394,-121.8950,AS29791 Internap Holding LLC,secure.onavo.com,95103,NaN
107.6.95.122,107.6.95.122,San Jose,California,US,"37.3394,-121.8950",America/Los_Angeles,United States,37.3394,-121.8950,AS29791 Internap Holding LLC,secure.onavo.com,95103,NaN
107.6.95.13,107.6.95.13,San Jose,California,US,"37.3394,-121.8950",America/Los_Angeles,United States,37.3394,-121.8950,AS29791 Internap Holding LLC,secure.onavo.com,95103,NaN
107.6.95.16,107.6.95.16,San Jose,California,US,"37.3394,-121.8950",America/Los_Angeles,United States,37.3394,-121.8950,AS29791 Internap Holding LLC,secure.onavo.com,95103,NaN
107.6.95.20,107.6.95.20,San Jose,California,US,"37.3394,-121.8950",America/Los_Angeles,United States,37.3394,-121.8950,AS29791 Internap Holding LLC,secure.onavo.com,95103,NaN


In [18]:
# Add/mix frequency data

dataBase["frequency"] = ipSize
dataBase.head(5)

,ip,city,region,country,loc,timezone,country_name,latitude,longitude,org,hostname,postal,bogon,frequency
101.199.108.59,101.199.108.59,Beijing,Beijing,CN,"39.9075,116.3972",Asia/Shanghai,China,39.9075,116.3972,NaN,NaN,NaN,NaN,32
101.226.167.226,101.226.167.226,Shanghai,Shanghai,CN,"31.2222,121.4581",Asia/Shanghai,China,31.2222,121.4581,AS4812 China Telecom (Group),NaN,NaN,NaN,1
107.178.37.48,107.178.37.48,San Jose,California,US,"37.3394,-121.8950",America/Los_Angeles,United States,37.3394,-121.8950,AS174 Cogent Communications,107-178-37-48.ip.cloudmosa.com,95103,NaN,194
107.178.38.221,107.178.38.221,San Jose,California,US,"37.3394,-121.8950",America/Los_Angeles,United States,37.3394,-121.8950,AS174 Cogent Communications,107-178-38-221.ip.cloudmosa.com,95103,NaN,176
107.21.14.116,107.21.14.116,Ashburn,Virginia,US,"39.0437,-77.4875",America/New_York,United States,39.0437,-77.4875,"AS14618 Amazon.com, Inc.",ec2-107-21-14-116.compute-1.amazonaws.com,20149,NaN,2


In [20]:
# Export to .csv so it can be visualised through Tableau:

dataBase.to_csv('IPs_data.csv')